# PyAcademy - VChu

Observed Trend 1: School spending per student has a negative impact on overall performance - more analysis should be considered
Observed Trend 2: When assessing how school size contributes to overall math and reading performance, Small(>1000 students) and Medium (1000-2000 students) schools tend to perform the same, however larger schools of 2000-5000 students have on average poorer performance than their Small and Medium School counterparts.
Observed Trend 3: Charter schools generally perform better overall in reading and math than district schools

In [114]:
import os
import pandas as pd

In [115]:
schools_csv = 'schools_complete.csv'
students_csv = 'students_complete.csv'
schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)

In [116]:
schools_df = schools_df.rename(columns = {
    "name" : "School Name",
    "type" : "School Type"
})
students_df = students_df.rename(columns = {
    "name" : "Student Name",
    "school": "School Name"
})
pyschool_df = pd.merge(schools_df, students_df, on = "School Name")

# District Summary

In [117]:
print ("District Summary")
pyschool_df["reading_score"] = pyschool_df["reading_score"].astype(float)
pyschool_df["math_score"] = pyschool_df["math_score"].astype(float)
pyschool_df["size"] = pyschool_df["size"].astype(float)
pyschool_df["budget"] = pyschool_df["budget"].astype(float)
pyschool_df.dtypes

District Summary


School ID          int64
School Name       object
School Type       object
size             float64
budget           float64
Student ID         int64
Student Name      object
gender            object
grade             object
reading_score    float64
math_score       float64
dtype: object

In [118]:
#Calculate Overall Summary totals
#Total Schools
totalschool = len(schools_df["School Name"].unique())
totalstudent = len(students_df["Student Name"].unique())
totalbudget = schools_df["budget"].sum()
avgmath = students_df["math_score"].mean()
avgread = students_df["reading_score"].mean()
mathpass = students_df.loc[(students_df["math_score"] >=60)]
mathpct = totalstudent / (len(mathpass)) *100
readpass = students_df.loc[(students_df["reading_score"]>=60)]
readpct = totalstudent / (len(readpass)) *100
overallpass = (readpct + mathpct)/2

In [119]:
districtsummary_df = pd.DataFrame({"Total Schools": [totalschool],
                                  "Total students": [totalstudent],
                                  "Total budget": [totalbudget],
                                  "Average Math Score": [avgmath],
                                  "Average Read Score": [avgread],
                                  "% Passing Math": [mathpct],
                                  "% Passing Reading":[readpct],
                                  "Overall Passing Rate": [overallpass]
                                  })
districtsummary_df = districtsummary_df.round(2)
districtsummary_df =[["Total Schools", "Total students", "Total budget", "Average Math Score", "% Passing Math", "Average Read Score", "% Passing Reading", "Overall Passing Rate"]]
districtsummary_df

[['Total Schools',
  'Total students',
  'Total budget',
  'Average Math Score',
  '% Passing Math',
  'Average Read Score',
  '% Passing Reading',
  'Overall Passing Rate']]

# School Summary

In [120]:
school_totalstud = pyschool_df.groupby("School Name")["Student Name"].count()
school_type = pyschool_df.groupby("School Name")["School Type"].unique().str.get(0)
school_budgetsum = pyschool_df.groupby("School Name")["budget"].unique().str.get(0)
school_budgetperstud = (school_budgetsum/school_totalstud)
school_avgmath = pyschool_df.groupby("School Name")["math_score"].mean()
school_avgread = pyschool_df.groupby("School Name")["reading_score"].mean()
school_mathpass = pyschool_df.loc[(students_df["math_score"]>=70)].groupby("School Name").count()
school_pctmathpass = ((school_mathpass["math_score"]/school_totalstud)*100)
school_readpass = pyschool_df.loc[(students_df["reading_score"]>=70)].groupby("School Name").count()
school_pctreadpass = ((school_readpass["reading_score"]/school_totalstud)*100)
school_overallpass = (school_pctreadpass+school_pctmathpass)/2

In [121]:
schoolsummary_df =pd.DataFrame({"Total students": school_totalstud,
                               "School Type":school_type,
                               "Total Budget": school_budgetsum,
                                "Total Budget/Student" : school_budgetperstud,
                                "Average Math Score": school_avgmath,
                                "% Passing Math": school_pctmathpass,
                                "Average Reading Score" : school_avgread,
                                "% Passing Reading": school_pctreadpass,
                                "Overall Passing Score": school_overallpass
                               })
schoolsummary_df = schoolsummary_df.round(0)
schoolsummary_df.head()

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Score,School Type,Total Budget,Total Budget/Student,Total students
School Name,,,,,,,,,
Bailey High School,67.0,82.0,77.0,81.0,74.0,District,3124928.0,628.0,4976
Cabrera High School,94.0,97.0,83.0,84.0,96.0,Charter,1081356.0,582.0,1858
Figueroa High School,66.0,81.0,77.0,81.0,73.0,District,1884411.0,639.0,2949
Ford High School,68.0,79.0,77.0,81.0,74.0,District,1763916.0,644.0,2739
Griffin High School,93.0,97.0,83.0,84.0,95.0,Charter,917500.0,625.0,1468


# Top Performing School (by Passing Rate)

In [122]:
#Top Performing Schools (By Passing Rate)
topschools_df = schoolsummary_df.sort_values(by="Overall Passing Score", ascending=False)
topschools_df.head(5)

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Score,School Type,Total Budget,Total Budget/Student,Total students
School Name,,,,,,,,,
Cabrera High School,94.0,97.0,83.0,84.0,96.0,Charter,1081356.0,582.0,1858
Griffin High School,93.0,97.0,83.0,84.0,95.0,Charter,917500.0,625.0,1468
Pena High School,95.0,96.0,84.0,84.0,95.0,Charter,585858.0,609.0,962
Shelton High School,94.0,96.0,83.0,84.0,95.0,Charter,1056600.0,600.0,1761
Thomas High School,93.0,97.0,83.0,84.0,95.0,Charter,1043130.0,638.0,1635


# Bottom Performing School (By Passing Rate)

In [123]:
#Bottom performing School by passing rate
bottomschool_df = schoolsummary_df.sort_values(by="Overall Passing Score")
bottomschool_df.head(5)

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Score,School Type,Total Budget,Total Budget/Student,Total students
School Name,,,,,,,,,
Figueroa High School,66.0,81.0,77.0,81.0,73.0,District,1884411.0,639.0,2949
Rodriguez High School,66.0,80.0,77.0,81.0,73.0,District,2547363.0,637.0,3999
Bailey High School,67.0,82.0,77.0,81.0,74.0,District,3124928.0,628.0,4976
Ford High School,68.0,79.0,77.0,81.0,74.0,District,1763916.0,644.0,2739
Hernandez High School,67.0,81.0,77.0,81.0,74.0,District,3022020.0,652.0,4635


# Math Scores by Grade

In [124]:
#Math Scores by Grade
math_grade_df=pyschool_df[["School Name", "grade","math_score"]]
mathbygrade=math_grade_df.groupby(["School Name","grade"])["math_score"].mean()
mathbygrade = mathbygrade.round()
mathbygrade.unstack().head()

grade,10th,11th,12th,9th
School Name,,,,
Bailey High School,77.0,78.0,76.0,77.0
Cabrera High School,83.0,83.0,83.0,83.0
Figueroa High School,77.0,77.0,77.0,76.0
Ford High School,78.0,77.0,76.0,77.0
Griffin High School,84.0,84.0,83.0,82.0


# Reading Score by Grade

In [125]:
#Reading Score by Grade
read_grade_df=pyschool_df[["School Name", "grade","reading_score"]]
readbygrade=read_grade_df.groupby(["School Name","grade"])["reading_score"].mean()
readbygrade=readbygrade.round()
readbygrade.unstack().head()

grade,10th,11th,12th,9th
School Name,,,,
Bailey High School,81.0,81.0,81.0,81.0
Cabrera High School,84.0,84.0,84.0,84.0
Figueroa High School,81.0,81.0,81.0,81.0
Ford High School,81.0,80.0,81.0,81.0
Griffin High School,84.0,84.0,84.0,83.0


# Scores by School Spending

In [126]:
#scorbyspend_df=schoolsummary_df[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", 
                                # "Overall Passing Score"]]
#scorbyspend_df["Total Budget/Student"]= pd.cut(schoolsummary_df["Total Budget/Student"], bins, labels=groupname)
bins = [550,600,650,700]
groupname = ["\$550-\$600", "\$600-\$650", "\$650-\$700"]

In [127]:
scorespend=pd.cut(schoolsummary_df["Total Budget/Student"], bins, labels=groupname)
scorespend_df = pd.DataFrame({ "Average Math Score": school_avgmath,
                                "% Passing Math": school_pctmathpass,
                                "Average Reading Score" : school_avgread,
                                "% Passing Reading": school_pctreadpass,
                                "Overall Passing Score": school_overallpass,
                                "Spending": scorespend})
scorespend_df.reset_index().drop("School Name",1)
spendingbins_df=scorespend_df.groupby("Spending").mean()
spendingbins_df.round()

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Score
Spending,,,,,
\$550-\$600,94.0,96.0,83.0,84.0,95.0
\$600-\$650,77.0,87.0,79.0,82.0,82.0
\$650-\$700,66.0,81.0,77.0,81.0,74.0


# Scores by School Size

In [128]:
bins = [0, 1000, 2000, 5000]
sizegroup =["Small (<1000)", "Medium (1000-2000)", "Large(2000-5000)"]

In [129]:
scores = pd.cut(schoolsummary_df["Total students"], bins, labels = sizegroup)
scores_df =pd.DataFrame({"Average Math Score": school_avgmath,
                                "% Passing Math": school_pctmathpass,
                                "Average Reading Score" : school_avgread,
                                "% Passing Reading": school_pctreadpass,
                                "Overall Passing Score": school_overallpass,
                                "School Size": scores
                             })
scores_df.reset_index().drop("School Name", 1)
scorebysize_df=scores_df.groupby("School Size").mean()
scorebysize_df.round()


,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Score
School Size,,,,,
Small (<1000),94.0,96.0,84.0,84.0,95.0
Medium (1000-2000),94.0,97.0,83.0,84.0,95.0
Large(2000-5000),70.0,83.0,78.0,81.0,76.0


# Scores by School Type

In [135]:
types_df = pd.DataFrame({"Average Math Score": school_avgmath,
                                "% Passing Math": school_pctmathpass,
                                "Average Reading Score" : school_avgread,
                                "% Passing Reading": school_pctreadpass,
                                "Overall Passing Score": school_overallpass,
                                 "School Type" : school_type
                             })
types_df.reset_index().drop("School Name",1)
scorebytype_df=types_df.groupby("School Type").mean()
scorebytype_df.round()

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Score
School Type,,,,,
Charter,94.0,97.0,83.0,84.0,95.0
District,67.0,81.0,77.0,81.0,74.0
